<a href="https://colab.research.google.com/github/glycodynamics/MSAnalyzer/blob/main/analyz_proteins_np_v1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import pandas as pd
import re

# Mount Google Drive
drive.mount('/content/drive')
data_dir = '/content/drive/My Drive/ms_treplicates'

# No change from here after
serum_dir = os.path.join(data_dir, 'serum')
sample_dir = os.path.join(data_dir, 'sample')

# Define the paths to the directories within your Google Drive
data_dir = '/content/drive/My Drive/ms_treplicates'
#serum_dir = '/content/drive/My Drive/ms_treplicates/serum'
#samples_dir = '/content/drive/My Drive/ms_treplicates/sample'

# Initialize serum data variables
serum_data_1 = pd.DataFrame()
serum_data_2 = pd.DataFrame()
serum_data_3 = pd.DataFrame()

# Define the UniProtID for normalization
UniProtID_Trypsin = 'P00761'  # Replace this with the actual UniProtID

# Read serum data, extract UniProt IDs, and store in respective variables
serum_files = [file for file in os.listdir(serum_dir) if file.endswith('.xlsx')]
for idx, file in enumerate(serum_files, start=1):
    serum_df = pd.read_excel(os.path.join(serum_dir, file))
    uni_prot_ids = [re.search(r'\|(.+?)\|', desc).group(1) for desc in serum_df['Description']]

    if idx == 1:
        serum_data_1 = serum_df.copy()
        serum_data_1['UniProtID'] = uni_prot_ids
        normalization_factor = serum_data_1.loc[serum_data_1['UniProtID'] == UniProtID_Trypsin, serum_data_1.columns[5]].values[0]
        serum_data_1['norm_Intensity'] = serum_data_1[serum_data_1.columns[5]] / normalization_factor
    elif idx == 2:
        serum_data_2 = serum_df.copy()
        serum_data_2['UniProtID'] = uni_prot_ids
        normalization_factor = serum_data_2.loc[serum_data_2['UniProtID'] == UniProtID_Trypsin, serum_data_2.columns[5]].values[0]
        serum_data_2['norm_Intensity'] = serum_data_2[serum_data_2.columns[5]] / normalization_factor
    elif idx == 3:
        serum_data_3 = serum_df.copy()
        serum_data_3['UniProtID'] = uni_prot_ids
        normalization_factor = serum_data_3.loc[serum_data_3['UniProtID'] == UniProtID_Trypsin, serum_data_3.columns[5]].values[0]
        serum_data_3['norm_Intensity'] = serum_data_3[serum_data_3.columns[5]] / normalization_factor

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create a list to store dictionaries
serum_data_list = []

# Iterate over unique UniProtIDs
unique_uniprot_ids = set(serum_data_1['UniProtID']).union(set(serum_data_2['UniProtID'])).union(set(serum_data_3['UniProtID']))
for uni_prot_id in unique_uniprot_ids:
    matching_rows = []
    norm_intensities = [None, None, None]  # Initialize with None values

    # Check if the UniProtID is present in each serum dataframe
    if uni_prot_id in serum_data_1['UniProtID'].values:
        matching_rows.append(serum_data_1[serum_data_1['UniProtID'] == uni_prot_id])
        norm_intensities[0] = serum_data_1[serum_data_1['UniProtID'] == uni_prot_id]['norm_Intensity'].values[0]
    if uni_prot_id in serum_data_2['UniProtID'].values:
        matching_rows.append(serum_data_2[serum_data_2['UniProtID'] == uni_prot_id])
        norm_intensities[1] = serum_data_2[serum_data_2['UniProtID'] == uni_prot_id]['norm_Intensity'].values[0]
    if uni_prot_id in serum_data_3['UniProtID'].values:
        matching_rows.append(serum_data_3[serum_data_3['UniProtID'] == uni_prot_id])
        norm_intensities[2] = serum_data_3[serum_data_3['UniProtID'] == uni_prot_id]['norm_Intensity'].values[0]

    # If UniProtID is found in more than one file, calculate mean and std deviation
    if len(matching_rows) > 1:
        combined_rows = pd.concat(matching_rows)
        mean_norm_intensity = combined_rows['norm_Intensity'].mean()
        std_dev_norm_intensity = combined_rows['norm_Intensity'].std()

        # Create a dictionary for the row
        new_row = {'UniProtID': uni_prot_id,
                   'norm_intensity_A': norm_intensities[0],
                   'norm_intensity_B': norm_intensities[1],
                   'norm_intensity_C': norm_intensities[2],
                   'avg_norm_intensity': mean_norm_intensity,
                   'std_norm_intensity': std_dev_norm_intensity}

        # Append the dictionary to the list
        serum_data_list.append(new_row)

# Create the serum_data DataFrame by passing the list of dictionaries
serum_data = pd.DataFrame(serum_data_list)
serum_data
# Print the resulting serum_data DataFrame

,UniProtID,norm_intensity_A,norm_intensity_B,norm_intensity_C,avg_norm_intensity,std_norm_intensity
0,P19827,0.153628,0.126244,0.024623,0.101498,0.067970
1,P00734,NaN,0.058833,0.016293,0.037563,0.030080
2,P00761,1.000000,1.000000,1.000000,1.000000,0.000000
3,P01591,NaN,0.243590,0.055882,0.149736,0.132729
4,P01834,2.208802,4.337248,2.109323,2.885124,1.258559
5,P02774,NaN,0.288407,0.068351,0.178379,0.155603
6,P02790,0.091662,0.320126,0.401296,0.271028,0.160550
7,P00751,NaN,0.309281,0.107137,0.208209,0.142938
8,P02760,NaN,0.136292,0.017666,0.076979,0.083881
9,Q14624,0.017245,0.080930,NaN,0.049088,0.045032


In [ ]:
import re
ref_UniProtID = 'P00761'
def calc_norm_intensity(input_df,ref_UniProtID):
    """
    Adds UniProtID and normalized intensity columns to the input DataFrame.

    Args:
        input_df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The input DataFrame with additional UniProtID and normalized intensity columns.
    """
    output_df = input_df.copy()

    output_df['UniProtID'] = [re.search(r'\|(.+?)\|', desc).group(1) for desc in output_df['Description']]

    intensity_p00761_rows = output_df[output_df['UniProtID'] == ref_UniProtID]

    if intensity_p00761_rows.empty:
        print("Intensity of the Trypsin is not present in one of the sample files" )
    else:
        intensity_p00761 = intensity_p00761_rows.iloc[0, output_df.columns.get_loc(output_df.columns[5])]
        output_df['norm_intensity'] = output_df[output_df.columns[5]] / intensity_p00761

    return output_df

In [ ]:
# Read sample data and apply calc_norm_intensity
sample_files = [file for file in os.listdir(sample_dir) if file.endswith('.xlsx')]
sample_data = {}
sample_IDs = []
sample_replicates = []

for file in sample_files:
    sample_name, replicate = file.split('.')[0].split('-')

    if sample_name not in sample_data:
        sample_data[sample_name] = {}

    sample_df = pd.read_excel(os.path.join(sample_dir, file))
    norm_intensity_df = calc_norm_intensity(sample_df,ref_UniProtID)
    sample_data[sample_name][replicate] = norm_intensity_df
    if sample_name not in sample_IDs:
        sample_IDs.append(sample_name)
# Print the sample data for a specific sample_name and replicate
desired_sample_name = 'H3'  # Replace with your actual sample name
desired_replicate = 'B'  # Replace with your actual replicate
sample_data[desired_sample_name][desired_replicate]

replicate_A_df = sample_data[sample_name]['A'].iloc[:, -2:]  # Print last two columns
sample_data

{'H2': {'C':     Protein\nRank                                        Description  \
  0              30  >sp|P01824|HV439_HUMAN Immunoglobulin heavy va...   
  1              31  >tr|A0A0B4J2B5|A0A0B4J2B5_HUMAN Immunoglobulin...   
  2              32  >sp|P00761|TRYP_PIG Trypsin OS=Sus scrofa OX=9...   
  3              34  >sp|P01591|IGJ_HUMAN Immunoglobulin J chain OS...   
  4              38  >sp|Q14624|ITIH4_HUMAN Inter-alpha-trypsin inh...   
  5              53  >sp|P01743|HV146_HUMAN Immunoglobulin heavy va...   
  6              64  >sp|P01602|KV105_HUMAN Immunoglobulin kappa va...   
  7              69  >sp|Q5VYS8|TUT7_HUMAN Terminal uridylyltransfe...   
  8              71  >sp|P59923|ZN445_HUMAN Zinc finger protein 445...   
  9              77  >sp|A8K8P3|SFI1_HUMAN Protein SFI1 homolog OS=...   
  10             97  >sp|Q96PC5|MIA2_HUMAN Melanoma inhibitory acti...   
  11            111  >sp|Q14674|ESPL1_HUMAN Separin OS=Homo sapiens...   
  12            119  >sp|P0

In [ ]:
def calc_relative_intensity(sample_data, sample_replicate):
  merged_data = pd.merge(serum_data, sample_replicate, on='UniProtID', how='inner')
  desired_columns = ['UniProtID','avg_norm_intensity', 'std_norm_intensity', 'norm_intensity']

  merged_data = merged_data[desired_columns]
  # Rename columns
  column_mapping = {
      'avg_norm_intensity': 'avg_int_serum',
      'std_norm_intensity': 'std_int_serum',
      'norm_intensity': 'norm_int_sample'
  }
  merged_data = merged_data.rename(columns=column_mapping)
  merged_data['relative_int_sample'] = merged_data['norm_int_sample'] / merged_data['avg_int_serum']
  return merged_data

In [ ]:
def find_common_UniProtIDs(sample_data, sample_name):
  replicate_A_df = sample_data[sample_name]['A'].iloc[:, -2:]  # Print last two columns
  replicate_B_df = sample_data[sample_name]['B'].iloc[:, -2:]  # Print last two columns
  replicate_C_df = sample_data[sample_name]['C'].iloc[:, -2:]  # Print last two columns

  # Assuming replicate_A_df, replicate_B_df, and replicate_C_df are three DataFrames containg UniProtIDs and norm_intensity in last two columns
  merged_df = pd.merge(replicate_A_df, replicate_B_df, on='UniProtID', how='outer', suffixes=('_A', '_B'))
  merged_df = pd.merge(merged_df, replicate_C_df.rename(columns={'norm_intensity': 'norm_intensity_C'}), on='UniProtID', how='outer')
  merged_df = merged_df.dropna(subset=['norm_intensity_A', 'norm_intensity_B', 'norm_intensity_C'], thresh=2)
  # Assuming merged_df is your merged DataFrame
  merged_df['avg_norm_intensity'] = merged_df[['norm_intensity_A', 'norm_intensity_B', 'norm_intensity_C']].apply(np.mean, axis=1)
  merged_df['std_norm_intensity'] = merged_df[['norm_intensity_A', 'norm_intensity_B', 'norm_intensity_C']].apply(np.std, axis=1)
  return merged_df

In [ ]:
import numpy as np
# Assuming relative_int_all is your dictionary containing relative intensity data frames
# Loop through each value in sample_IDs
relative_int_all=pd.DataFrame()
for val in sample_IDs:
    relative_int = find_common_UniProtIDs(sample_data, val)
    relative_int.drop(columns=['norm_intensity_A', 'norm_intensity_B', 'norm_intensity_C'], inplace=True)
    relative_int_all[val] = relative_int

# Merge the DataFrames based on 'UniProtID' using an outer join
merged_int_all_df = None  # Placeholder for the merged DataFrame

for key, df in relative_int_all.items():
    suffix = f'_{key}' # Add suffix only for subsequent DataFrames
    if merged_int_all_df is None:
        # If this is the first DataFrame, set merged_df to it
        merged_int_all_df = df.copy().rename(columns={
            'avg_norm_intensity': f'avg_norm_intensity{suffix}',
            'std_norm_intensity': f'std_norm_intensity{suffix}'
        })
    else:
         df_renamed = df.rename(columns={
            'avg_norm_intensity': f'avg_norm_intensity{suffix}',
            'std_norm_intensity': f'std_norm_intensity{suffix}'
        })
         merged_int_all_df = pd.merge(merged_int_all_df, df_renamed, on='UniProtID', how='outer')
        # Merge the current DataFrame with the existing merged_df

#        merged_df = pd.merge(merged_df, df, on='UniProtID', how='outer', suffixes=('', f'_{key}'))


ValueError: ignored

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def calculate_log10_df(dataframe):
    # Create a new DataFrame to store the log10 transformed values
    log10_transformed_df = dataframe.copy()

    # Iterate through columns and apply log10 transformation to numeric values
    for col in dataframe.columns:
        if pd.api.types.is_numeric_dtype(dataframe[col]):
            log10_transformed_df[col] = np.where(
                dataframe[col].notna(),
                np.log10(dataframe[col]),
                dataframe[col]
            )

    return log10_transformed_df


def calculate_log2_df(dataframe):
    # Create a new DataFrame to store the log10 transformed values
    log2_transformed_df = dataframe.copy()

    # Iterate through columns and apply log10 transformation to numeric values
    for col in dataframe.columns:
        if pd.api.types.is_numeric_dtype(dataframe[col]):
            log2_transformed_df[col] = np.where(
                dataframe[col].notna(),
                np.log2(dataframe[col]),
                dataframe[col]
            )

    return log2_transformed_df

merged_int_all_df_log10 = calculate_log10_df(merged_int_all_df)
merged_int_all_df_log2 = calculate_log2_df(merged_int_all_df)
merged_int_all_df_log10

NameError: ignored

In [ ]:
import os
import pandas as pd

# Write data to a file
data_dir = '/content/drive/My Drive/ms_treplicates'
file_path = os.path.join(data_dir, 'Results.xlsx')
excel_writer = pd.ExcelWriter(file_path)

# Write each dataframe to a different worksheet
merged_int_all_df.to_excel(excel_writer, sheet_name='Int', index=False)
merged_int_all_df_log2.to_excel(excel_writer, sheet_name='log2', index=False)
merged_int_all_df_log10.to_excel(excel_writer, sheet_name='log10', index=False)
# Close the Excel writer to properly save the file
excel_writer.close()


In [ ]:
merged_int_all_df_log2 = merged_int_all_df_log10[merged_int_all_df_log2['UniProtID'] != 'P00761']
merged_int_all_df_log10 = merged_int_all_df_log10[merged_int_all_df_log10['UniProtID'] != 'P00761']

# Extract UniProtID column
prot_ids = merged_int_all_df_log10['UniProtID']

# Get column names for average intensity and standard deviation
avg_columns = [col for col in merged_int_all_df_log10.columns if 'avg_norm_intensity' in col]
std_columns = [col.replace('avg_', 'std_') for col in avg_columns]

# Create a figure and axis
fig, ax = plt.subplots(figsize=(30, 6))

# Calculate the number of bars and the bar width
num_bars = len(prot_ids)
bar_width = 0.4

# Loop through each pair of avg and std columns
for i, (avg_col, std_col) in enumerate(zip(avg_columns, std_columns)):
    avg_intensities = merged_int_all_df_log10[avg_col]
    std_intensities = np.abs(merged_int_all_df_log10[std_col])

    # Calculate x-coordinates for bars
    x = np.arange(num_bars) + i * (bar_width+10)

    # Plot avg_norm_intensity as bars
    bars = ax.bar(x, avg_intensities, width=bar_width, label=avg_col, alpha=0.7)

    # Calculate x-coordinates for error bars at the center of the bars
    error_x = x + bar_width/2

    # Plot std_norm_intensity as error bars with caps at the center of the bars
    error_caps = ax.errorbar(error_x, avg_intensities, yerr=std_intensities, lw =1, linestyle='None', color='black', capsize=2)

# Set labels and title
ax.set_xlabel('UniProtID')
ax.set_ylabel('Log10 Intensity')
ax.set_title('UniProtID vs Average Intensities with Error Bars')
ax.set_xticks(np.arange(num_bars) + (len(avg_columns) - 1) * bar_width / 2)
ax.set_xticklabels(prot_ids, rotation=90, fontsize=10)

ax.legend()

# Show the plot
plt.tight_layout()
plt.ylim([-5, 5])
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

merged_int_all_df_log2 = merged_int_all_df_log10[merged_int_all_df_log2['UniProtID'] != 'P00761']
merged_int_all_df_log10 = merged_int_all_df_log10[merged_int_all_df_log10['UniProtID'] != 'P00761']

fig, ax = plt.subplots(figsize=(40, 6))
barWidth = 0.2

br1 = np.arange(len(merged_int_all_df_log10['avg_norm_intensity_H2']))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]

# Make the plot
plt.bar(br1, merged_int_all_df_log10['avg_norm_intensity_H2'], yerr = np.abs(merged_int_all_df_log10['std_norm_intensity_H2']), color ='r', width = barWidth, lw =1,
        edgecolor ='black', label ='B2')
plt.bar(br2, merged_int_all_df_log10['avg_norm_intensity_H3'], yerr = np.abs(merged_int_all_df_log10['std_norm_intensity_H3']), color ='g', width = barWidth, lw =1,
        edgecolor ='black', label ='B3')
plt.bar(br3, merged_int_all_df_log10['avg_norm_intensity_D1'], yerr = np.abs(merged_int_all_df_log10['std_norm_intensity_D1']), color ='b', width = barWidth, lw =1,
        edgecolor ='black', label ='D1')
plt.xticks([r + barWidth for r in range(len(merged_int_all_df_log10['UniProtID']))],
         merged_int_all_df_log10['UniProtID'], rotation=90, fontsize=16)


ax.set_xlabel('UniProtID')
ax.set_ylabel('Log10 Intensity')
ax.set_title('UniProtID vs Average Intensities with Error Bars')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

merged_int_all_df_log2 = merged_int_all_df_log10[merged_int_all_df_log2['UniProtID'] != 'P00761']
#merged_int_all_df_log10 = merged_int_all_df_log10[merged_int_all_df_log10['UniProtID'] != 'P00761']

fig, ax = plt.subplots(figsize=(40, 6))
barWidth = 0.2

br1 = np.arange(len(merged_int_all_df_log2['avg_norm_intensity_H2']))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]

# Make the plot
plt.bar(br1, merged_int_all_df_log2['avg_norm_intensity_H2'], yerr = np.abs(merged_int_all_df_log2['std_norm_intensity_H2']), color ='r', width = barWidth, lw =1,
        edgecolor ='black', label ='B2')
plt.bar(br2, merged_int_all_df_log2['avg_norm_intensity_H3'], yerr = np.abs(merged_int_all_df_log2['std_norm_intensity_H3']), color ='g', width = barWidth, lw =1,
        edgecolor ='black', label ='B3')
plt.bar(br3, merged_int_all_df_log2['avg_norm_intensity_D1'], yerr = np.abs(merged_int_all_df_log2['std_norm_intensity_D1']), color ='b', width = barWidth, lw =1,
        edgecolor ='black', label ='D1')
plt.xticks([r + barWidth for r in range(len(merged_int_all_df_log2['UniProtID']))],
         merged_int_all_df_log2['UniProtID'], rotation=90, fontsize=16)


ax.set_xlabel('UniProtID')
ax.set_ylabel('Log10 Intensity')
ax.set_title('UniProtID vs Average Intensities with Error Bars')
plt.grid(True)
plt.show()

NameError: ignored